In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from fpdf import FPDF

# === CONFIGURATION ===
folder_path = '/Users/ajfoeckler/Downloads/chicago_dockhounds_games'
save_folder_path = '/Users/ajfoeckler/Downloads/dockhounds_pitchers_vs_chicago'
os.makedirs(save_folder_path, exist_ok=True)

# === LOAD ALL CSVs FIRST (then filter) ===
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
data_frames = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, low_memory=False)
    data_frames.append(df)

original_data = pd.concat(data_frames, ignore_index=True)

# Clean up common string cols to avoid silent mismatches
for c in ['Pitcher','PitcherTeam','Batter','KorBB','PitchCall','PlayResult','AutoPitchType']:
    if c in original_data.columns:
        original_data[c] = original_data[c].astype(str).str.strip()

# Keep only 2025 games (if desired)
original_data['Date'] = pd.to_datetime(original_data['Date'], errors='coerce')
original_data = original_data.dropna(subset=['Date'])
original_data = original_data[original_data['Date'].dt.year == 2025]

# === NOW filter DockHounds pitchers ===
dockhounds_pitchers = original_data[original_data['PitcherTeam'] == 'LAK_COU10'].copy()

# Optional diagnostics
print(f"Rows after 2025 filter: {len(original_data):,}")
print("PitcherTeam counts:\n", original_data['PitcherTeam'].value_counts(dropna=False).to_string())
print(f"DockHounds rows (LAK_COU10): {len(dockhounds_pitchers):,}")
print("Sample pitchers:", dockhounds_pitchers['Pitcher'].dropna().unique()[:10])

# === SERIES-SPECIFIC HITTERS TO INCLUDE FOR STRIKE ZONE ===
hitters_of_interest = [
    'Hopkins, TJ', 'Kusiak, Henry', 'Teter, Jacob', 'Stroup, Dusty', 'Maiben, Jacob',
    'Pruitt, Reggie', 'Sisco, Chance', 'Turbo, Johnni', 'Clark, Tripp', 'Williams, Jaylyn'
]  # add more as needed

# === PITCH COLORS ===
pitch_colors = {
    'Four-Seam': 'red',
    'Sinker': 'orange',
    'Slider': 'blue',
    'Curveball': 'purple',
    'Changeup': 'green',
    'Cutter': 'cyan',
    'Splitter': 'pink',
    'Sweeper': 'teal',
}

def _build_sort_cols(df):
    pref = ['GameID','Inning','Top/Bottom','PAofInning','PitchofPA','PitchNo','RelTime']
    return [c for c in pref if c in df.columns]

def create_pitcher_report(pitcher_name, pitcher_data):
    pdf = FPDF()
    pdf.add_page()

    # ===== HEADER =====
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, f'{pitcher_name} - Opposing Hitter Matchup Report', ln=True, align='C')
    pdf.ln(3)

    # ===== MATCHUP SUMMARY TABLE (PAGE 1) — PA LOGIC (terminal pitch per PA) =====
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(0, 8, 'Matchup Summary:', ln=True)

    summary_rows = []
    hitters_faced = pitcher_data['Batter'].dropna().unique()

    for hitter in hitters_faced:
        md = pitcher_data[pitcher_data['Batter'] == hitter].copy()
        if md.empty:
            continue

        # Identify PA keys present in data
        pa_key_cols = [c for c in ['GameID','Inning','Top/Bottom','PAofInning','Batter'] if c in md.columns]
        if not pa_key_cols:
            # Fallback: approximate PA boundaries by batter changes within a game
            md = md.sort_values(_build_sort_cols(md) or ['GameID'])
            seq_key = (md['Batter'].shift(1) != md['Batter']).cumsum()
            md['__PA_FALLBACK__'] = seq_key
            pa_key_cols = [c for c in ['GameID','__PA_FALLBACK__','Batter'] if c in md.columns]

        sort_cols = _build_sort_cols(md) or pa_key_cols + (['PitchNo'] if 'PitchNo' in md.columns else [])
        md = md.sort_values(sort_cols)

        # Take the terminal pitch of each PA
        pa_last = md.groupby(pa_key_cols, dropna=False, as_index=False).tail(1).copy()

        # Normalize columns we’ll use
        pa_last['KorBB'] = pa_last.get('KorBB', '').astype(str)
        pa_last['PitchCall'] = pa_last.get('PitchCall', '').astype(str)
        pa_last['PlayResult'] = pa_last.get('PlayResult', '').astype(str)

        # Flags from terminal pitch only
        is_walk = pa_last['KorBB'].isin(['Walk','IntentionalWalk'])
        is_hbp = pa_last['PitchCall'].eq('HitByPitch')
        is_sac = pa_last['PlayResult'].isin(['Sacrifice','SacrificeBunt','SacrificeFly'])
        is_interf = pa_last['PlayResult'].isin(['Interference','CatcherInterference'])

        is_k = pa_last['KorBB'].eq('Strikeout') | pa_last['PlayResult'].eq('Strikeout')
        is_hit = pa_last['PlayResult'].isin(['Single','Double','Triple','HomeRun'])
        is_hr  = pa_last['PlayResult'].eq('HomeRun')

        # AB: exclude BB, HBP, sacs, interference
        is_ab = ~(is_walk | is_hbp | is_sac | is_interf)

        PA = int(len(pa_last))
        AB = int(is_ab.sum())
        H  = int((is_hit & is_ab).sum())
        HR = int((is_hr  & is_ab).sum())
        K  = int(is_k.sum())
        BB = int(is_walk.sum())
        AVG = (H / AB) if AB > 0 else 0.0

        summary_rows.append({
            'Hitter': hitter, 'PA': PA, 'AB': AB, 'H': H,
            'K': K, 'BB': BB, 'HR': HR, 'AVG_AGAINST': f'{AVG:.3f}'
        })

    summary_df = pd.DataFrame(summary_rows).sort_values(['PA','Hitter'], ascending=[False, True])

    # Compact table styling
    cols = ['Hitter', 'PA', 'AB', 'H', 'K', 'BB', 'HR', 'AVG_AGAINST']
    col_widths = [55, 16, 16, 16, 16, 16, 16, 28]
    row_h = 6

    pdf.set_font('Arial', 'B', 10)
    for w, c in zip(col_widths, cols):
        pdf.cell(w, row_h, c, border=1, align='C')
    pdf.ln(row_h)

    pdf.set_font('Arial', '', 10)
    for _, r in summary_df.iterrows():
        for w, c in zip(col_widths, cols):
            pdf.cell(w, row_h, str(r[c]), border=1, align='C')
        pdf.ln(row_h)

    pdf.ln(2)

    # ===== PITCH USAGE TABLE (THIS PITCHER) =====
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(0, 8, 'Pitch Usage with Velocity (This Pitcher):', ln=True)

    usage_summary = pitcher_data.groupby('AutoPitchType', dropna=True).agg(
        Count=('PitchNo', 'count'),
        AvgVelo=('RelSpeed', 'mean')
    ).reset_index()
    total = usage_summary['Count'].sum() if not usage_summary.empty else 0
    usage_summary['Usage%'] = (usage_summary['Count'] / total * 100).round(1) if total > 0 else 0.0

    cols_usage = ['AutoPitchType', 'Count', 'Usage%', 'AvgVelo']
    col_widths_usage = [70, 26, 26, 26]
    row_h_usage = 6

    pdf.set_font('Arial', 'B', 10)
    for w, c in zip(col_widths_usage, cols_usage):
        pdf.cell(w, row_h_usage, c, border=1, align='C')
    pdf.ln(row_h_usage)

    pdf.set_font('Arial', '', 10)
    for _, r in usage_summary.iterrows():
        vals = [
            str(r['AutoPitchType']),
            str(int(r['Count'])),
            f"{float(r['Usage%']):.1f}" if isinstance(r['Usage%'], (int, float)) else str(r['Usage%']),
            f"{float(r['AvgVelo']):.1f}" if pd.notna(r['AvgVelo']) else ''
        ]
        for w, v in zip(col_widths_usage, vals):
            pdf.cell(w, row_h_usage, v, border=1, align='C')
        pdf.ln(row_h_usage)

    # ===== STRIKE ZONES vs HITTERS OF INTEREST =====
    faced_interest = [h for h in hitters_of_interest if h in hitters_faced]
    plot_paths = []

    for hitter in faced_interest:
        sub_data = pitcher_data[pitcher_data['Batter'] == hitter]

        plt.figure(figsize=(6, 6))
        zone = {'top': 3.67, 'bottom': 1.52, 'left': -0.83, 'right': 0.83}
        plt.plot([zone['left'], zone['right'], zone['right'], zone['left'], zone['left']],
                 [zone['bottom'], zone['bottom'], zone['top'], zone['top'], zone['bottom']], 'k-')

        for pt in sub_data['AutoPitchType'].dropna().unique():
            pitch_sub = sub_data[sub_data['AutoPitchType'] == pt]
            plt.scatter(
                pitch_sub['PlateLocSide'], pitch_sub['PlateLocHeight'],
                alpha=0.7, color=pitch_colors.get(pt, 'gray'), edgecolor='black',
                label=f"{pt} ({len(pitch_sub)} | {pitch_sub['RelSpeed'].mean():.1f} mph)"
            )

            if 'PitchCall' in pitch_sub.columns:
                _call_display = {'FoulBallNotFieldable': 'FoulBall', 'BallCalled': 'Ball'}
                _offsets = [(3, 3), (-3, 3), (3, -3), (-3, -3)]
                _has_extras = all(c in pitch_sub.columns for c in ['TaggedHitType','PlayResult','ExitSpeed'])

                for j, row in pitch_sub.iterrows():
                    x = row.get('PlateLocSide', None); y = row.get('PlateLocHeight', None)
                    if pd.isna(x) or pd.isna(y): continue
                    raw_call = str(row.get('PitchCall', ''))
                    call_display = _call_display.get(raw_call, raw_call)
                    label_text = call_display
                    if raw_call == 'InPlay' and _has_extras:
                        hit_type = str(row.get('TaggedHitType', '') or '')
                        play_res = str(row.get('PlayResult', '') or '')
                        ev = row.get('ExitSpeed', None)
                        ev_str = f"EV {float(ev):.1f}" if pd.notna(ev) else ""
                        parts = [raw_call, "|", hit_type, play_res, ev_str]
                        label_text = " ".join([p for p in parts if p])
                    dx, dy = _offsets[j % len(_offsets)]
                    plt.annotate(label_text, (x, y), textcoords="offset points", xytext=(dx, dy),
                                 ha="left", va="bottom", fontsize=6, alpha=0.9,
                                 bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.6))

        plt.xlim(-2, 2); plt.ylim(0, 5)
        plt.xlabel('Horizontal Location (ft)'); plt.ylabel('Vertical Location (ft)')
        plt.title(f'{pitcher_name} vs {hitter}')
        plt.legend(fontsize=7); plt.grid(True, linestyle='--', alpha=0.5)

        fname = f"{pitcher_name.replace(',', '')}_{hitter.replace(',', '')}_loc.png"
        plot_file = os.path.join(save_folder_path, fname)
        plt.savefig(plot_file, dpi=150, bbox_inches='tight'); plt.close()
        plot_paths.append((hitter, plot_file))

    # ===== PLACE ALL STRIKE-ZONE IMAGES (4 per page; unlimited pages) =====
    if plot_paths:
        margin_x, margin_y = 15, 28
        img_w, img_h = 90, 90
        dx, dy = 95, 110

        for i in range(0, len(plot_paths), 4):
            pdf.add_page()
            batch = plot_paths[i:i+4]
            for j, (hitter, path) in enumerate(batch):
                col = j % 2; row = j // 2
                x = margin_x + col * dx; y = margin_y + row * dy
                pdf.set_xy(x, y - 6); pdf.set_font('Arial', 'B', 10)
                pdf.cell(img_w, 6, f'{pitcher_name} vs {hitter}', align='C')
                pdf.image(path, x=x, y=y, w=img_w, h=img_h)

    # ===== SAVE PDF =====
    pdf_file = os.path.join(save_folder_path, f'{pitcher_name.replace(",", "")}_vs_chicago.pdf')
    pdf.output(pdf_file)

# === GENERATE REPORTS FOR DOCKHOUNDS PITCHERS ===
unique_pitchers = dockhounds_pitchers['Pitcher'].dropna().unique()
for pitcher in unique_pitchers:
    p_df = dockhounds_pitchers[dockhounds_pitchers['Pitcher'] == pitcher]
    if p_df.empty:
        continue
    create_pitcher_report(pitcher, p_df)

print('All pitcher reports generated successfully.')


/Users/ajfoeckler/opt/anaconda3/lib/python3.9/site-packages/fpdf/__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


Rows after 2025 filter: 5,539
PitcherTeam counts:
 CHI_DOG      2886
LAK_COU10    2653
DockHounds rows (LAK_COU10): 2,653
Sample pitchers: ['Jefferson, Chris' 'Fenlong, Connor' 'Philip, Beau' 'Cantleberry, Jake'
 'Berrier, Logan' 'Conine, Brett' 'Torres, Eric' 'Lobstein, Kyle'
 'Gsellman, Robert' 'Hansel, Luke']


/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:65: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:66: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'{pitcher_name} - Opposing Hitter Matchup Report', ln=True, align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:70: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:71: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pd

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:65: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:66: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'{pitcher_name} - Opposing Hitter Matchup Report', ln=True, align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:70: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:71: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pd

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:65: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:66: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'{pitcher_name} - Opposing Hitter Matchup Report', ln=True, align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:70: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:71: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pd

...

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:65: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:66: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'{pitcher_name} - Opposing Hitter Matchup Report', ln=True, align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:70: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:71: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pd

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:65: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:66: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'{pitcher_name} - Opposing Hitter Matchup Report', ln=True, align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:70: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:71: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pd

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:65: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:66: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'{pitcher_name} - Opposing Hitter Matchup Report', ln=True, align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:70: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:71: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pd

/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:235: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 10)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:65: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:66: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'{pitcher_name} - Opposing Hitter Matchup Report', ln=True, align='C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_31680/2437371396.py:70: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  p

All pitcher reports generated successfully.
